In [1]:
!pip install transformers torch torchvision torchaudio accelerate datasets requests -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but 

In [2]:
import requests

f = open("pg2591.txt", "w")
f.write(requests.get("https://www.gutenberg.org/cache/epub/2591/pg2591.txt").text)
f.close()

In [3]:
storiesTitles = [
    "THE GOLDEN BIRD", "HANS IN LUCK", "JORINDA AND JORINDEL", "THE TRAVELLING MUSICIANS", "OLD SULTAN", "THE STRAW, THE COAL, AND THE BEAN",
    "BRIAR ROSE", "THE DOG AND THE SPARROW", "THE TWELVE DANCING PRINCESSES", "THE FISHERMAN AND HIS WIFE", "THE WILLOW-WREN AND THE BEAR",
    "THE FROG-PRINCE", "CAT AND MOUSE IN PARTNERSHIP", "THE GOOSE-GIRL", "THE ADVENTURES OF CHANTICLEER AND PARTLET", "RAPUNZEL",
    "FUNDEVOGEL", "THE VALIANT LITTLE TAILOR", "HANSEL AND GRETEL", "THE MOUSE, THE BIRD, AND THE SAUSAGE", "MOTHER HOLLE",
    "LITTLE RED-CAP [LITTLE RED RIDING HOOD]", "THE ROBBER BRIDEGROOM", "TOM THUMB", "RUMPELSTILTSKIN", "CLEVER GRETEL", "THE OLD MAN AND HIS GRANDSON",
    "THE LITTLE PEASANT", "FREDERICK AND CATHERINE", "SWEETHEART ROLAND", "SNOWDROP", "THE PINK", "CLEVER ELSIE", "THE MISER IN THE BUSH",
    "ASHPUTTEL", "THE WHITE SNAKE", "THE WOLF AND THE SEVEN LITTLE KIDS", "THE QUEEN BEE", "THE ELVES AND THE SHOEMAKER", "THE JUNIPER-TREE",
    "THE TURNIP", "CLEVER HANS", "THE THREE LANGUAGES", "THE FOX AND THE CAT", "THE FOUR CLEVER BROTHERS", "LILY AND THE LION",
    "THE FOX AND THE HORSE", "THE BLUE LIGHT", "THE RAVEN", "THE GOLDEN GOOSE", "THE WATER OF LIFE", "THE TWELVE HUNTSMEN", "THE KING OF THE GOLDEN MOUNTAIN",
    "DOCTOR KNOWALL", "THE SEVEN RAVENS", "THE WEDDING OF MRS FOX", "THE SALAD", "THE STORY OF THE YOUTH WHO WENT FORTH TO LEARN WHAT FEAR WAS",
    "KING GRISLY-BEARD", "IRON HANS", "CAT-SKIN", "SNOW-WHITE AND ROSE-RED"
]
stories = {title: [] for title in storiesTitles}
f = open("pg2591.txt", "r")
title = None
for line in f.readlines():
    text = line.strip()
    if "*** END OF THE PROJECT GUTENBERG EBOOK GRIMMS' FAIRY TALES ***" == text:
        break
    if text in stories:
        title = text
        continue
    if title:
        stories[title].append(text)
f.close()
dataset = {
    "text": []
}
for story in stories:
    dataset["text"].append(" ".join(list(filter(lambda x: x!="", stories[story]))))
dataset["text"] = list(filter(lambda x: x!="", dataset["text"]))

In [4]:
import os

download_dir = './huggingface/transformers'
os.environ["HF_HOME"] = download_dir
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

In [5]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = Dataset.from_dict(dataset)
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [6]:
block_size = 128
def group_texts(examples):
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  total_length = (total_length // block_size) * block_size
  result = {
    k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
    for k, t in concatenated_examples.items()
  }
  result["labels"] = result["input_ids"].copy()
  return result

lm_datasets = tokenized_datasets.map(
  group_texts,
  batched=True,
  batch_size=1000,
  num_proc=4,
)

Map (num_proc=4):   0%|          | 0/62 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoConfig, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
  "test_trainer",
  evaluation_strategy = "no",
  learning_rate=2e-5,
  weight_decay=0.01,
  num_train_epochs=200
  # use_mps_device=True
)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=lm_datasets,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
500,2.700500
1000,2.178000
1500,1.771300
2000,1.388900
2500,1.063600
3000,0.808100
3500,0.612900
4000,0.476700
4500,0.372800
5000,0.302000


TrainOutput(global_step=12400, training_loss=0.5474389713041244, metrics={'train_runtime': 4267.47, 'train_samples_per_second': 23.246, 'train_steps_per_second': 2.906, 'total_flos': 6480042393600000.0, 'train_loss': 0.5474389713041244, 'epoch': 200.0})

In [12]:
trainer.save_model("my-model")

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import textwrap

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("./my-model")
text = "Once upon a time, there was a king"
inputs = tokenizer(text, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=110)
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print("\n".join(textwrap.wrap(" ".join(generated_text), 80)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Once upon a time, there was a king who had twelve beautiful daughters. They
slept in twelve beds all in one room; and when they went to bed, the doors were
shut and locked up; but every morning their shoes were found to be quite worn
through as if they had been danced in all night; and yet nobody could find out
how it happened, or where they had been. Then the king made it known to all the
land, that if any person could discover the secret, and find out where it was
that the princesses danced in the night, he should have the one he
